In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
120112,Thiago Alcántara do Nascimento,9,3,0,Liverpool,Manchester United,1.909495,1.999295,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120113,Thiago Emiliano da Silva,9,2,90,Chelsea,Norwich,1.962383,0.410877,True,6,...,0.0,1.0,0.0,85.0,90.0,94.4,61.0,0.0,0.0,1
120114,Thomas Partey,9,3,90,Arsenal,Aston Villa,1.241777,0.909682,True,8,...,0.0,2.0,0.0,42.0,52.0,80.8,39.0,1.0,2.0,1
120115,Timothy Castagne,9,2,90,Leicester City,Brentford,1.207841,0.440627,False,2,...,0.0,0.0,0.0,26.0,32.0,81.3,23.0,1.0,1.0,1
120116,Timo Werner,9,4,0,Chelsea,Norwich,1.962383,0.410877,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120117,Tim Krul,9,1,90,Norwich,Chelsea,0.410877,1.962383,False,1,...,0.0,0.0,0.0,39.0,51.0,76.5,22.0,0.0,0.0,1
120118,Tino Livramento,9,2,90,Southampton,Burnley,0.562381,0.320196,True,9,...,0.1,5.0,0.0,56.0,79.0,70.9,54.0,3.0,4.0,1
120119,Tobi Omole,9,2,0,Tottenham Hotspur,West Ham United,1.535970,0.780656,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120120,Toby Alderweireld,9,2,0,Tottenham Hotspur,West Ham United,1.535970,0.780656,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120121,Todd Cantwell,9,3,0,Norwich,Chelsea,0.410877,1.962383,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

10

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,10,4,0,Brighton and Hove Albion,Liverpool,0.566036,1.908443,False,0,...,9552,2,2,0.0,-279,957,1236,0,2021-10-30T14:00:00Z,2122
1,Aaron Cresswell,10,2,90,West Ham United,Aston Villa,0.780226,0.909181,False,2,...,1308512,4,1,8.0,93320,122335,29015,0,2021-10-31T16:30:00Z,2122
2,Aaron Lennon,10,3,0,Burnley,Brentford,0.320019,0.440385,True,0,...,1324,1,3,0.0,-55,106,161,0,2021-10-30T14:00:00Z,2122
3,Aaron Ramsdale,10,1,90,Arsenal,Leicester City,1.241094,1.207176,False,10,...,568492,2,0,0.0,31397,56736,25339,0,2021-10-30T11:30:00Z,2122
4,Aaron Ramsey,10,3,0,Aston Villa,West Ham United,0.909181,0.780226,True,0,...,16005,4,1,0.0,-420,760,1180,0,2021-10-31T16:30:00Z,2122
5,Aaron Wan-Bissaka,10,2,90,Manchester United,Tottenham Hotspur,2.007004,1.535124,False,7,...,394597,3,0,26.0,-44241,3340,47581,0,2021-10-30T16:30:00Z,2122
6,Abdoulaye Doucouré,10,3,0,Everton,Wolverhampton Wanderers,0.944310,0.845970,False,0,...,236270,1,2,0.0,-200354,747,201101,0,2021-11-01T20:00:00Z,2122
7,Adama Traoré,10,3,0,Wolverhampton Wanderers,Everton,0.845970,0.944310,True,0,...,192891,1,2,0.0,-23121,6484,29605,0,2021-11-01T20:00:00Z,2122
8,Adam Armstrong,10,4,90,Southampton,Watford,0.562071,0.320460,False,5,...,204055,1,0,43.0,-40253,8355,48608,0,2021-10-30T14:00:00Z,2122
9,Adam Forshaw,10,3,13,Leeds,Norwich,0.569780,0.410651,False,1,...,1246,2,1,0.0,-18,80,98,0,2021-10-31T14:00:00Z,2122


In [10]:
df_latest.shape

(623, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,Kasper Schmeichel,Leicester City,1.0
1,Jonny Evans,Leicester City,1.0
2,Marc Albrighton,Leicester City,0.0
3,Nampalys Mendy,Leicester City,0.0
4,Danny Ward,Leicester City,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Antonio Rüdiger
Mateo Kovačić
Jorginho
César Azpilicueta
Andreas Christensen
Romelu Lukaku
Marcos Alonso
Kai Havertz
Thiago Silva
Trevoh Chalobah
Mason Mount
Reece James
Timo Werner
Ben Chilwell
N'Golo Kanté
Callum Hudson-Odoi
Ruben Loftus-Cheek
Hakim Ziyech
Kepa Arrizabalaga
Malang Sarr
Christian Pulisic
Saúl Ñíguez
Ross Barkley
Emerson Palmieri
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Virgil van Dijk
Mohamed Salah
Sadio Mané
Alisson
Jordan Henderson
Trent Alexander-Arnold
Joël Matip
Diogo Jota
Andrew Robertson
Fabinho
James Milner
Naby Keïta
Roberto Firmino
Curtis Jones
Kostas Tsimikas
Ibrahima Konaté
Harvey Elliott
Thiago Alcántara
Alex Oxlade-Chamberlain
Caoimhín Kelleher
Joe Gomez
Neco Williams
Takumi Minamino
Divock Origi
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
João Cancelo
Rúben Dias
Ederson


In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Edouard Mendy,2021-08-14,Chelsea,Premier League,1,0,0,20,0,...,0.0,0,0,18,20,90.0,10,0,0,Edouard Mendy
1,2021-2022,Edouard Mendy,2021-08-22,Chelsea,Premier League,2,0,0,46,0,...,0.0,0,0,37,43,86.0,30,0,0,Edouard Mendy
2,2021-2022,Edouard Mendy,2021-08-28,Chelsea,Premier League,3,0,0,47,0,...,0.0,0,0,26,40,65.0,9,0,0,Edouard Mendy
3,2021-2022,Edouard Mendy,2021-09-11,Chelsea,Premier League,4,0,0,42,1,...,0.0,0,0,30,36,83.3,17,0,0,Edouard Mendy
4,2021-2022,Edouard Mendy,2021-09-25,Chelsea,Premier League,6,0,0,38,0,...,0.0,0,0,28,34,82.4,16,0,0,Edouard Mendy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,2021-2022,Adam Idah,2021-10-02,Norwich,Premier League,7,0,0,5,1,...,0.0,0,0,1,2,50.0,3,0,0,Adam Idah
2772,2021-2022,Adam Idah,2021-10-16,Norwich,Premier League,8,0,0,2,0,...,0.0,1,0,2,2,100.0,3,0,0,Adam Idah
2773,2021-2022,Adam Idah,2021-10-31,Norwich,Premier League,10,0,0,3,4,...,0.0,0,0,3,3,100.0,2,0,0,Adam Idah
2774,2021-2022,Bali Mumba,2021-08-21,Norwich,Premier League,2,0,0,19,12,...,0.0,0,0,7,10,70.0,9,0,1,Bali Mumba


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['Lyanco', 'Przemysław Płacheta']

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['Lyanco'] = 'Lyanco Evangelista Silveira Neves Vojnovic'
# match_list['Przemysław Płacheta'] = 'Przemyslaw Placheta'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [30]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
# df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

In [41]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [42]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
117918,Harry Kane,6,4,90,Tottenham Hotspur,Arsenal,1.551043,1.220584,False,2,...,0,0,0,10,13,76.9,12,0,1,1
118530,Harry Kane,7,4,90,Tottenham Hotspur,Aston Villa,1.535970,0.909682,True,2,...,0.3,3,0,20,28,71.4,31,1,4,1
119145,Harry Kane,8,4,90,Tottenham Hotspur,Newcastle United,1.535970,0.540785,False,12,...,0.8,3,1,23,25,92,20,1,2,1
119762,Harry Kane,9,4,90,Tottenham Hotspur,West Ham United,1.535970,0.780656,False,2,...,0,1,0,21,26,80.8,24,2,4,1
120381,Harry Kane,10,4,90,Tottenham Hotspur,Manchester United,1.535124,2.007004,True,2,...,0.0,0,0,15,26,57.7,17,0,0,1


In [43]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
120778,Yerson Mosquera Valdelamar,10,2,0,Wolverhampton Wanderers,Everton,0.845970,0.944310,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120779,Yoane Wissa,10,3,0,Brentford,Burnley,0.440385,0.320019,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120780,Youri Tielemans,10,3,90,Leicester City,Arsenal,1.207176,1.241094,True,2,...,0.2,7,0,54,65,83.1,55,0,0,1
120781,Yves Bissouma,10,3,59,Brighton and Hove Albion,Liverpool,0.566036,1.908443,False,1,...,0.0,0,0,24,27,88.9,26,2,3,1
120782,Zack Steffen,10,1,0,Manchester City,Crystal Palace,2.378671,0.549407,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
120783,Álvaro Fernández,10,1,90,Brentford,Burnley,0.440385,0.320019,False,1,...,0.0,0,0,33,46,71.7,30,0,0,1
120784,Çaglar Söyüncü,10,2,90,Leicester City,Arsenal,1.207176,1.241094,True,1,...,0.0,2,0,82,90,91.1,72,1,1,1


In [44]:
df_latest_further.shape

(623, 57)

In [45]:
df_train_new.shape

(120785, 59)

In [46]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [47]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [48]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
100,Southampton,Aston Villa,2021-11-05T20:00:00Z,11,0
101,Manchester United,Manchester City,2021-11-06T12:30:00Z,11,1
102,Brentford,Norwich,2021-11-06T15:00:00Z,11,2
103,Chelsea,Burnley,2021-11-06T15:00:00Z,11,3
104,Crystal Palace,Wolverhampton Wanderers,2021-11-06T15:00:00Z,11,4
105,Brighton and Hove Albion,Newcastle United,2021-11-06T17:30:00Z,11,5
106,Arsenal,Watford,2021-11-07T14:00:00Z,11,6
107,Everton,Tottenham Hotspur,2021-11-07T14:00:00Z,11,7
108,Leeds,Leicester City,2021-11-07T14:00:00Z,11,8
109,West Ham United,Liverpool,2021-11-07T16:30:00Z,11,9


In [49]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [50]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,20,7
1,11,1,Manchester United,Manchester City,2021-11-06T12:30:00Z,1,43
2,11,2,Brentford,Norwich,2021-11-06T15:00:00Z,94,45
3,11,3,Chelsea,Burnley,2021-11-06T15:00:00Z,8,90
4,11,4,Crystal Palace,Wolverhampton Wanderers,2021-11-06T15:00:00Z,31,39


In [51]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [52]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
6,Granit Xhaka,0,3,7,49,3,Xhaka
8,Héctor Bellerín,0,2,9,48,3,Bellerín
17,Lucas Torreira,0,3,18,45,3,Torreira
...,...,...,...,...,...,...,...
607,Owen Otasowie,0,3,442,45,39,Otasowie
609,Andreas Söndergaard,0,1,444,40,39,Söndergaard
611,Yerson Mosquera Valdelamar,0,2,446,45,39,Mosquera
616,Patrick Cutrone,0,4,523,50,39,Cutrone


In [53]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [54]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [55]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,11,0,Aston Villa,Southampton,2021-11-05T20:00:00Z,Jed Steer,1,40,None,Steer,False
1,11,0,Aston Villa,Southampton,2021-11-05T20:00:00Z,Conor Hourihane,3,49,0,Hourihane,False
2,11,0,Aston Villa,Southampton,2021-11-05T20:00:00Z,Emiliano Martínez,1,55,100,Martínez,False
3,11,0,Aston Villa,Southampton,2021-11-05T20:00:00Z,Björn Engels,2,45,0,Engels,False
4,11,0,Aston Villa,Southampton,2021-11-05T20:00:00Z,Bertrand Traoré,3,57,0,Traoré,False


In [56]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Theo Walcott,3,58,100,Walcott,True
1,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Fraser Forster,1,44,None,Forster,True
2,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Alex McCarthy,1,45,None,McCarthy,True
3,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Oriol Romeu Vidal,3,45,None,Romeu,True
4,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Nathan Redmond,3,59,None,Redmond,True


In [57]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [58]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [59]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Theo Walcott,3,5.8,1.00,Walcott,True,0.562071,0.909181,2122
1,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Fraser Forster,1,4.4,1.00,Forster,True,0.562071,0.909181,2122
2,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Alex McCarthy,1,4.5,1.00,McCarthy,True,0.562071,0.909181,2122
3,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Oriol Romeu Vidal,3,4.5,1.00,Romeu,True,0.562071,0.909181,2122
4,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Nathan Redmond,3,5.9,1.00,Redmond,True,0.562071,0.909181,2122
5,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Jack Stephens,2,4.4,0.00,Stephens,True,0.562071,0.909181,2122
6,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Stuart Armstrong,3,5.8,1.00,S.Armstrong,True,0.562071,0.909181,2122
7,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,James Ward-Prowse,3,6.3,1.00,Ward-Prowse,True,0.562071,0.909181,2122
8,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Kyle Walker-Peters,2,4.8,1.00,Walker-Peters,True,0.562071,0.909181,2122
9,11,0,Southampton,Aston Villa,2021-11-05T20:00:00Z,Jan Bednarek,2,4.4,1.00,Bednarek,True,0.562071,0.909181,2122


In [60]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')